# Chapter 6
## Data Loading, Storage, and File Formats

In [5]:
import pandas as pd

In [67]:
import numpy as np

In [6]:
data = pd.read_csv('/Users/roethelchristine/dev/2dot1-WesWrangling/examples/macrodata.csv')

## 6.1 Reading and Writing Data in Text Format

Pandas features a number of functions for reading tabular data as a DataFrame object. See below for overview of the mechanics of these functions, which are meant to convert text data into a DataFrame. The optional arguments for these functions may fall into a few categories:

<i>Indexing</i>: Can treat one or more columns as the returned DataFrame, and whether to get column names from the file, the user, or not at all.

<i>Type inference and data coversation</i>: This includes the user-defined value conversations and custom list of missing value markers.

<i>Datetime parsing</i>: Includes combining capability, including combining date and time information spread over multiple columns into a single column in the result.

<i>Iterating</i>: Support for iterating over chunks of very large files.

<i>Unclean data issues</i>: Skipping rows or a footer, comments, or other minor things like numeric data with thousands separated by commas.

In [7]:
!cat examples/ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

<b>Since this is comma-delimited, we can use read_csv to read it into a DataFrame:

In [8]:
df = pd.read_csv('examples/ex1.csv')

In [9]:
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


<b>We could also use read_table and specified the delimiter:

In [10]:
pd.read_table('examples/ex1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


<b>Another example, where a file will not always have a header row:

In [11]:
!cat examples/ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [12]:
pd.read_csv('examples/ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [14]:
pd.read_csv('examples/ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [15]:
names = ['a', 'b', 'c', 'd', 'message']

<b>Suppose you wanted the message column to be the index of the returned DataFrame. You can either indicate you want the column at index 4 or named 'message' using the index_col argument:

In [16]:
pd.read_csv('examples/ex2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


<b>In the event that you want to form a hierarchical index from multiple columns, pass a list of column numbers or names:

In [17]:
!cat examples/csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [18]:
parsed = pd.read_csv('examples/csv_mindex.csv', index_col=['key1', 'key2'])

In [19]:
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

<b>In some cases, a table might not have a fixed delimiter, using whitespace or some other pattern to separate fields. Consider a text file that looks like this:

In [20]:
list(open('examples/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

<b>While you could do some munging by hand, the fields here are separated by a variable amount of whitespace. In these cases, you can pass a regular expression as a delimiter for read_table. This can be expressed by the regular expression \s+, so we have then:

In [21]:
result = pd.read_table('examples/ex3.txt', sep='\s+')

In [22]:
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


<b>Because there was one fewer column name than the number of data rows, read_table infers that the first column should be the DataFrame’s index in this special case.

<b>The parser functions have many additional arguments to help you handle the wide variety of exception file formats that occur. For example, you can skip the first, third, and fourth rows of a file with skiprows:

In [23]:
!cat examples/ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [25]:
pd.read_csv('examples/ex4.csv', skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


<b>Handling missing values is an important and frequently nuanced part of the file pars‐ ing process. Missing data is usually either not present (empty string) or marked by some sentinel value. By default, pandas uses a set of commonly occurring sentinels, such as NA and NULL:

In [24]:
!cat examples/ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo

In [26]:
result = pd.read_csv('examples/ex5.csv')

In [27]:
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [28]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


<b>The na_values option can take either a list or set of strings to consider missing values:

In [29]:
result = pd.read_csv('examples/ex5.csv', na_values=['NULL'])

In [30]:
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


<b>Different NA sentinels can be specified for each column in a dict:

In [31]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}

In [33]:
pd.read_csv('examples/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


## Reading Text Files in Pieces

In [34]:
pd.options.display.max_rows = 10

In [35]:
result = pd.read_csv('examples/ex6.csv')

In [36]:
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


<b>If you want to only read a small number of rows (avoiding reading the entire file), specify that with nrows:

In [37]:
pd.read_csv('examples/ex6.csv', nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


<b>To read a file in pieces, specify a chunksize as a number of rows:

In [38]:
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)

In [39]:
chunker

<b>The TextParser object returned by read_csv allows you to iterate over the parts of the file according to the chunksize. For example, we can iterate over ex6.csv, aggre‐ gating the value counts in the 'key' column like so:

In [50]:
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)

In [51]:
tot = pd.Series([])

/var/folders/5v/j_knnv350nsfj2p76j0_nrcm0000gq/T/ipykernel_12780/1115746282.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  tot = pd.Series([])


In [52]:
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)

In [53]:
tot = tot.sort_values(ascending=False)

In [54]:
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

## Writing Data to Text Format

In [55]:
data = pd.read_csv('examples/ex5.csv')

In [56]:
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [57]:
data.to_csv('examples/out.csv')

In [58]:
!cat examples/out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


<b>Other delimiters can be used, of course (writing to sys.stdout so it prints the text result to the console):

In [59]:
import sys

In [60]:
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


<b>Missing values appear as empty strings in the output. You might want to denote them by some other sentinel value:

In [61]:
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [62]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [63]:
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


In [64]:
dates = pd.date_range('1/1/2000', periods=7)

In [68]:
ts = pd.Series(np.arange(7), index=dates)

In [69]:
ts.to_csv('examples/tseries.csv')

In [70]:
!cat examples/tseries.csv

,0
2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


## Working with Delimited Formats

In [71]:
!cat examples/ex7.csv

"a","b","c"
"1","2","3"
"1","2","3"


In [72]:
import csv

In [75]:
f = open('examples/ex7.csv')

In [76]:
reader = csv.reader(f)

In [77]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [93]:
with open('examples/ex7.csv') as f:
    lines = list(csv.reader(f))

In [94]:
header, values = lines[0], lines[1:]

In [95]:
data_dict = {h: v for h, v in zip(header, zip(*values))}

In [96]:
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

In [97]:
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL

In [98]:
reader = csv.reader(f, dialect=my_dialect)

ValueError: I/O operation on closed file.

In [99]:
reader = csv.reader(f, delimiter='|')

ValueError: I/O operation on closed file.

In [100]:
with open('mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect) 
    writer.writerow(('one', 'two', 'three')) 
    writer.writerow(('1', '2', '3')) 
    writer.writerow(('4', '5', '6')) 
    writer.writerow(('7', '8', '9'))

## JSON Data

In [103]:
obj = """
    {"name": "Wes",
     "places_lived": ["United States", "Spain", "Germany"],
     "pet": null,
     "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
                  {"name": "Katie", "age": 38,
                   "pets": ["Sixes", "Stache", "Cisco"]}]
} 
"""

In [104]:
import json

In [105]:
result = json.loads(obj)

In [106]:
result

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

In [107]:
asjson = json.dumps(result)

In [109]:
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])

In [110]:
siblings

,name,age
0,Scott,30
1,Katie,38


In [111]:
!cat examples/example.json

[{"a": 1, "b": 2, "c": 3},
 {"a": 4, "b": 5, "c": 6},
 {"a": 7, "b": 8, "c": 9}]


In [112]:
data = pd.read_json('examples/example.json')

In [113]:
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [114]:
print(data.to_json())

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}


In [115]:
print(data.to_json(orient='records'))

[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]


## XML and HTML: Web Scraping

In [117]:
pip install lxml

You should consider upgrading via the '/usr/local/Cellar/jupyterlab/3.1.14_1/libexec/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [118]:
pip install beautifulsoup4 html5lib

     |████████████████████████████████| 97 kB 2.0 MB/s eta 0:00:01
     |████████████████████████████████| 112 kB 6.5 MB/s eta 0:00:01
You should consider upgrading via the '/usr/local/Cellar/jupyterlab/3.1.14_1/libexec/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [121]:
tables = pd.read_html('examples/fdic_failed_bank_list.html')

In [122]:
len(tables)

1

In [123]:
failures = tables[0]

In [124]:
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


In [128]:
close_timestamps = pd.to_datetime(failures['Closing Date'])

In [129]:
close_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
       ... 
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, Length: 15, dtype: int64

### Parsing XML with lxml.objectify

XML (eXtensible Markup Language) is another common structured data format sup‐ porting hierarchical, nested data with metadata.

Using lxml.objectify, we parse the file and get a reference to the root node of the XML file with getroot:

In [130]:
from lxml import objectify

In [133]:
path = 'examples/mta_perf/Performance_MNR.xml'

In [134]:
parsed = objectify.parse(open(path))

FileNotFoundError: [Errno 2] No such file or directory: 'examples/mta_perf/Performance_MNR.xml'

In [135]:
root = parsed.getroot()

AttributeError: 'DataFrame' object has no attribute 'getroot'

<b>root.INDICATOR returns a generator yielding each <INDICATOR> XML element. For each record, we can populate a dict of tag names (like YTD_ACTUAL) to data values (excluding a few tags):

In [136]:
data = []

In [137]:
skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ', 'DESIRED_CHANGE', 'DECIMAL_PLACE']

In [138]:
for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

NameError: name 'root' is not defined

In [139]:
perf = pd.DataFrame(data)

In [140]:
perf.head()

""


In [141]:
from io import StringIO

In [142]:
tag = '<a href ="http://ww.google.com">Google</a>'

In [143]:
root = objectify.parse(StringIO(tag)).getroot()

In [144]:
root

<Element a at 0x11e8b1800>

In [145]:
root.get('href')

'http://ww.google.com'

In [146]:
root.text

'Google'

## 6.2 Binary Data Formats

One of the easiest ways to store data (also known as serialization) efficiently in binary format is using Python’s built-in pickle serialization. pandas objects all have a to_pickle method that writes the data to disk in pickle format:

In [147]:
frame = pd.read_csv('examples/ex1.csv')

In [148]:
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [149]:
frame.to_pickle('examples/frame_pickle')

In [150]:
pd.read_pickle('examples/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


<i>bcolz</i>: A Compressable column-oriented binary format based on the Blosc compression library.

<i>Feather</i>: A cross-language column-oriented file format I designed with the R program‐ ming community’s Hadley Wickham. Feather uses the Apache Arrow columnar memory format.

## Using HDF5 Format

HDF5 is a well-regarded file format intended for storing large quantities of scientific array data. It is available as a C library, and it has interfaces available in many other languages, including Java, Julia, MATLAB, and Python. The “HDF” in HDF5 stands for hierarchical data format. Each HDF5 file can store multiple datasets and support‐ ing metadata. Compared with simpler formats, HDF5 supports on-the-fly compres‐ sion with a variety of compression modes, enabling data with repeated patterns to be stored more efficiently. HDF5 can be a good choice for working with very large data‐ sets that don’t fit into memory, as you can efficiently read and write small sections of much larger arrays.


While it’s possible to directly access HDF5 files using either the PyTables or h5py libraries, pandas provides a high-level interface that simplifies storing Series and DataFrame object. The HDFStore class works like a dict and handles the low-level details:

In [171]:
frame = pd.DataFrame({'a': np.random.randn(100)})

In [172]:
store = pd.HDFStore('mydata.h5')

ImportError: Missing optional dependency 'tables'.  Use pip or conda to install tables.

In [173]:
store['obj1'] = frame

UsageError: Unknown variable '[obj1]'


In [160]:
store['obj1_col'] = frame['a']

UsageError: Unknown variable '[obj1_col]'


In [161]:
store

Stored variables and their in-db values:


In [174]:
store['obj1']

UsageError: Unknown variable '[obj1]'


In [175]:
store.put('obj2', frame, format='table')

NameError: name 'store' is not defined

In [176]:
store.select('obj2', where=['index >= 10 and index <= 15'])

NameError: name 'store' is not defined

In [177]:
store.close()

NameError: name 'store' is not defined

In [178]:
frame.to_hdf('mydata.h5', 'obj3', format='table')

ImportError: Missing optional dependency 'tables'.  Use pip or conda to install tables.

In [179]:
pd.read_hdf('mydata.h5', 'obj3', where=['index < 5'])

FileNotFoundError: File mydata.h5 does not exist

## Reading Microsoft Excel Files

pandas also supports reading tabular data stored in Excel 2003 (and higher) files using either the ExcelFile class or pandas.read_excel function. Internally these tools use the add-on packages xlrd and openpyxl to read XLS and XLSX files, respec‐ tively. You may need to install these manually with pip or conda.

<b>To use ExcelFile, create an instance by passing a path to an xls or xlsx file:

In [183]:
xlsx = pd.ExcelFile('examples/ex1.xlsx')

In [182]:
pip install openpyxl

     |████████████████████████████████| 242 kB 2.8 MB/s eta 0:00:01
You should consider upgrading via the '/usr/local/Cellar/jupyterlab/3.1.14_1/libexec/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


<b>Data stored in a sheet can then be read into DataFrame with parse:

In [184]:
pd.read_excel(xlsx, 'Sheet1')

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


<b>If you are reading multiple sheets in a file, then it is faster to create the ExcelFile, but you can also simply pass the filename to pandas.read_excel:

In [187]:
frame = pd.read_excel(xlsx, 'Sheet1')

In [188]:
frame

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [190]:
writer = pd.ExcelWriter('examples/ex2.xlsx')

<b>ITo write pandas data to Excel format, you must first create an ExcelWriter, then write data to it using pandas objects’ to_excel method:

In [191]:
frame.to_excel(writer, 'Sheet1')

In [192]:
writer.save()

<b>You can also pass a file path to to_excel and avoid the ExcelWriter:

In [193]:
frame.to_excel('examples/ex2.xlsx')

## 6.3 Interacting with Web APIs

Many websites have public APIs providing data feeds via JSON or some other format. There are a number of ways to access these APIs from Python; one easy-to-use method that I recommend is the requests package.

To find the last 30 GitHub issues for pandas on GitHub, we can make a GET HTTP request using the add-on requests library:

In [194]:
import requests

In [195]:
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'

In [196]:
resp = requests.get(url)

In [197]:
resp

<Response [200]>

<b>The Response object’s json method will return a dictionary containing JSON parsed into native Python objects:

In [198]:
data = resp.json()

In [199]:
data[0]['title']

'REF: rolling benchmarks to reduce redundant benchmarks'

<b>Each element in data is a dictionary containing all of the data found on a GitHub issue page (except for the comments). We can pass data directly to DataFrame and extract fields of interest:

In [200]:
issues = pd.DataFrame(data, columns=['number', 'title', 'labels', 'state'])

In [201]:
issues

,number,title,labels,state
0,44475,REF: rolling benchmarks to reduce redundant be...,"[{'id': 732775912, 'node_id': 'MDU6TGFiZWw3MzI...",open
1,44474,BUG: Tick + np.timedelta64,[],open
2,44471,BUG: fix timedelta floordiv with scalar float ...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
3,44470,"BUG: Segmentation-fault during a ""custom"" roll...","[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
4,44467,ENH: Implement reading the output of a MDX que...,"[{'id': 76812, 'node_id': 'MDU6TGFiZWw3NjgxMg=...",open
...,...,...,...,...
25,44426,TYP: improve typing for DataFrame.to_string,"[{'id': 1280988427, 'node_id': 'MDU6TGFiZWwxMj...",open
26,44424,ENH: pd.Series.shift to alternatively accept a...,"[{'id': 76812, 'node_id': 'MDU6TGFiZWw3NjgxMg=...",open
27,44421,BUG: pd.read_sql() fails on self join when han...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
28,44414,BUG: DataFrame attributes are lost after speci...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open


<b>With a bit of elbow grease, you can create some higher-level interfaces to common web APIs that return DataFrame objects for easy analysis.

## 6.4 Interacting with Databases

In a business setting, most data may not be stored in text or Excel files. SQL-based relational databases (such as SQL Server, PostgreSQL, and MySQL) are in wide use, and many alternative databases have become quite popular. The choice of database is usually dependent on the performance, data integrity, and scalability needs of an application.

Loading data from SQL into a DataFrame is fairly straightforward, and pandas has some functions to simplify the process. As an example, I’ll create a SQLite database using Python’s built-in sqlite3 driver:

In [202]:
import sqlite3

In [203]:
query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
c REAL, d INTEGER
);"""

In [205]:
con = sqlite3.connect('mydata.sqlite')

In [206]:
con.execute(query)

In [207]:
con.commit()

<b>Then, insert a few rows of data:

In [209]:
data = [('Atlanta', 'Georgia', 1.25, 6),
         ('Tallahassee', 'Florida', 2.6, 3),
         ('Sacramento', 'California', 1.7, 5)]

In [210]:
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"

In [211]:
con.executemany(stmt, data)

In [212]:
con.commit()

<b>Most Python SQL drivers (PyODBC, psycopg2, MySQLdb, pymssql, etc.) return a list of tuples when selecting data from a table:

In [214]:
cursor = con.execute('select * from test')

In [215]:
rows = cursor.fetchall()

In [216]:
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

<b>You can pass the list of tuples to the DataFrame constructor, but you also need the column names, contained in the cursor’s description attribute:

In [217]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [218]:
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


This is quite a bit of munging that you’d rather not repeat each time you query the database. The SQLAlchemy project is a popular Python SQL toolkit that abstracts away many of the common differences between SQL databases. pandas has a read_sql function that enables you to read data easily from a general SQLAlchemy connection. Here, we’ll connect to the same SQLite database with SQLAlchemy and read data from the table created before:

In [220]:
pip install sqlalchemy

     |████████████████████████████████| 1.5 MB 2.4 MB/s eta 0:00:01
     |████████████████████████████████| 92 kB 1.4 MB/s eta 0:00:01
You should consider upgrading via the '/usr/local/Cellar/jupyterlab/3.1.14_1/libexec/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [224]:
import sqlalchemy as sqla

In [225]:
db = sqla.create_engine('sqlite:///mydata.sqlite')

In [226]:
pd.read_sql('select * from test', db)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
